In [20]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

You should consider upgrading via the 'C:\Users\beeman_lucas\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\beeman_lucas\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\beeman_lucas\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\beeman_lucas\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [21]:
import gym 
import random

Cartpole-V1 is the enviroment provided by the Gym library
states are the possible states something can be in, in this case, it is "dead or alive"
actions are the possible actions the NN can make, this case is 2, left or right

In [22]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [23]:
actions

2

Testing the enviroment

In [24]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:24.0
Episode:2 Score:12.0
Episode:3 Score:32.0
Episode:4 Score:9.0
Episode:5 Score:18.0
Episode:6 Score:76.0
Episode:7 Score:14.0
Episode:8 Score:40.0
Episode:9 Score:18.0
Episode:10 Score:21.0


In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

Adam is an optimizer build for compilation of the model

In [26]:
Adam = tf.optimizers.Adam(learning_rate=1e-3)

This is the model of the neural network
There is one input layer, 2 hidden layers, and 1 output layers

In [27]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [28]:
model = build_model(states, actions)

In [43]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 4)                 0         
                                                                 
 dense_9 (Dense)             (None, 24)                120       
                                                                 
 dense_10 (Dense)            (None, 24)                600       
                                                                 
 dense_11 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [44]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

This is what we use to build and compile the NN

In [45]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=1000)
    return dqn

we compile with the Adam optimizer and a metric known as mean absolute error

In [46]:
dqn = build_agent(model, actions)
dqn.compile(optimizer=Adam, metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: 1.0000
269 episodes - episode_reward: 37.160 [9.000, 136.000] - loss: 0.039 - mae: 2.915 - mean_q: 6.003

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 104s 10ms/step - reward: 1.0000
190 episodes - episode_reward: 52.616 [10.000, 247.000] - loss: 0.059 - mae: 3.335 - mean_q: 6.954

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 103s 10ms/step - reward: 1.0000
109 episodes - episode_reward: 90.743 [11.000, 447.000] - loss: 0.082 - mae: 3.765 - mean_q: 7.923

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 108s 11ms/step - reward: 1.0000
104 episodes - episode_reward: 96.913 [12.000, 341.000] - loss: 0.118 - mae: 4.188 - mean_q: 8.867

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 109s 11ms/step - reward: 1.0000
done, t

scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

In [36]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 25.000, steps: 25
Episode 2: reward: 27.000, steps: 27
Episode 3: reward: 28.000, steps: 28
Episode 4: reward: 25.000, steps: 25
Episode 5: reward: 22.000, steps: 22
Episode 6: reward: 25.000, steps: 25
Episode 7: reward: 23.000, steps: 23
Episode 8: reward: 28.000, steps: 28
Episode 9: reward: 25.000, steps: 25
Episode 10: reward: 24.000, steps: 24
Episode 11: reward: 25.000, steps: 25
Episode 12: reward: 23.000, steps: 23
Episode 13: reward: 24.000, steps: 24
Episode 14: reward: 27.000, steps: 27
Episode 15: reward: 24.000, steps: 24


In [ ]:
save the NN

In [37]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [38]:
del model
del dqn
del env

In [40]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(optimizer=Adam, metrics=['mae'])

In [41]:
dqn.load_weights('dqn_weights.h5f')

In [42]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 23.000, steps: 23
Episode 2: reward: 45.000, steps: 45
Episode 3: reward: 26.000, steps: 26
Episode 4: reward: 58.000, steps: 58
Episode 5: reward: 28.000, steps: 28
